#### Context Managers
There is always some setup and tear down notion that you want to combine.
```
with open('contextmanager.py') as f:
    pass
```
e.g. open `contextmanager.py` file, do something, close file.<br>
This concept is particularly important when writing/with IO buffers as you want to make sure you write to disk before closing the file.

#### Example: Tables

In [2]:
from sqlite3 import connect

In [3]:
with connect('test.db') as conn: # connect method as a context manager
    cur = conn.cursor()
    cur.execute('create table points(x int, y int)')
    cur.execute('insert into points (x, y) values (1, 1)')
    cur.execute('insert into points (x, y) values (2, 1)')
    for row in cur.execute('select sum(x * y) from points'):
        print(row)
    cur.execute('drop table points')

(3,)


In the above example, the connection to the database is a context manager i.e. we open an connection, commit some statements and then close the connection.<br>
<b>But</b> there is another context manager going on here --> _create and drop a table_

NOTE: Naturally this could be done with a transaction which has the right semantics
but for this example, we'll pretend we don't have this capability and need to create and drop table.

### Remeber!
<b>there is always some top-level syntax or function and a corresponding `__method__` that implements it.</b>
```
x + y --> __add__
init x --> __init__
repr(x) --> __repr__
x() --> __call__
```
So a context manager is essentially...
```
x = contextmanager().__enter__()
try:
    pass
finally:
    x.__exit__()
```

In [4]:
class TempTable: # temp table as a context manager
    def __init__(self, cur):
        self.cur = cur
    def __enter__(self):
        self.cur.execute('create table points(x int, y int)')
    def __exit__(self, *args):
        self.cur.execute('drop table points')

with connect('test.db') as conn: # connect method as a context manager
    cur = conn.cursor()
    with TempTable(cur):
        cur.execute('insert into points (x, y) values (1, 1)')
        cur.execute('insert into points (x, y) values (2, 1)')
        for row in cur.execute('select sum(x * y) from points'):
            print(row)

(3,)


NOTE: Context managers have a clear and unambiguous metaphor behind them.<br>
So let's look at something interesting with the temp table context manager.

```
class TempTable:
    def __init__(self, cur):
        self.cur = cur
    def __enter__(self):
        self.cur.execute('create table points(x int, y int)')
    def __exit__(self, *args):
        self.cur.execute('drop table points')
```

<b>The enter should always be called before the exit</b> --> Use a generator!<br>
NOTE: Generators force sequencing. They are a mechanism by which you can create code that interleaves with other code and also enforces interleaving (co-routines).

#### Temp Table as a Generator
- We want to a process that creates a table, accepts some user code and then drops the table: CONTEXT MANAGER 
- We want to enforce the sequencing of creating a table, executing some statements and then dropping a table: GENERATOR
- So we want to create a generator and wrap it in a context manager!

In [10]:
def temptable(cur):
    cur.execute('create table points(x int, y int)')
    yield
    cur.execute('drop table points')

class contextmanager:
    def __init__(self, cur):
        self.cur = cur
    def __enter__(self):
        self.gen = temptable(self.cur)
        next(self.gen)
    def __exit__(self, *args):
        next(self.gen, None)

with connect('test.db') as conn:
    cur = conn.cursor()
    with contextmanager(cur):
        cur.execute('insert into points (x, y) values (1, 1)')
        cur.execute('insert into points (x, y) values (2, 1)')
        for row in cur.execute('select sum(x * y) from points'):
            print(row)

(3,)


#### Generalizing the contextmanager Class
The context manager class is hard coded, so let's make it more generalized

In [7]:
class contextmanager:
    def __init__(self, gen):
        self.gen = gen
    def __call__(self, *args, **kwargs): # call the generator with args, kwargs
        self.args, self.kwargs = args, kwargs
        return self
    def __enter__(self): # iterate over the generator
        self.gen_inst = self.gen(*self.args, **self.kwargs)
        next(self.gen_inst)
    def __exit__(self, *args): # end the generator
        next(self.gen_inst, None)

@contextmanager # wrap the generator in a context manager
def temptable(cur):
    cur.execute('create table points(x int, y int)')
    yield
    cur.execute('drop table points')
    
# temptable = contextmanager(temptable)

with connect('test.db') as conn:
    cur = conn.cursor()
    with temptable(cur):
        cur.execute('insert into points (x, y) values (1, 1)')
        cur.execute('insert into points (x, y) values (2, 1)')
        for row in cur.execute('select sum(x * y) from points'):
            print(row)

(3,)


#### Contextmanager has already been written!

In [8]:
from contextlib import contextmanager

@contextmanager
def temptable(cur):
    cur.execute('create table points(x int, y int)')
    try:
        yield
    finally:
        cur.execute('drop table points')

with connect('test.db') as conn: # the connect method is a context manager too
    cur = conn.cursor()
    with temptable(cur):
        cur.execute('insert into points (x, y) values (1, 1)')
        cur.execute('insert into points (x, y) values (2, 1)')
        for row in cur.execute('select sum(x * y) from points'):
            print(row)

(3,)


## Summary:

3 fundamental features, mechanisms and a clear reason on how to use them:
1. <b>context manager</b>: 
    - setup & tear down pairing. ensures tear-down if setup occurred.
2. <b>generator</b>: 
    - merely syntax that enforces sequencing and interleaving
    - prevents eager loading i.e. take a long computation and break it up into chunks for the user to process

NOTE: A Context Manager requires sequencing and interleaving i.e. setup, wait for user code, then tear down. i.e. generator. We also need something to adapt the generator to the data model i.e. wrapping the generator into a context manager (Decorator!)
3. <b>decorator</b>: wrapping the generator into a context manager.


Code that has clarity for when and where a feature should be use - I have this pattern and python has this mechanism. No additional mechanism or protocols etc. the language is specific to the purpose